# Projet 2 - Interaction avec des APIs

Ce projet vise à introduire au requêtage d’API et à la manipulation de
données qui en sont issues, en utilisant des fonctions afin de faciliter
la reproductibilité des analyses.

## Contexte du projet

Une API (Interface de Programmation d’Application) est un ensemble de
règles et de spécifications que les applications peuvent suivre pour
communiquer entre elles. Elle permet à votre code d’**accéder à des
fonctionnalités externes ou à des données**, comme celles de bases de
données météorologiques ou de services de localisation.

Lorsqu’on parle de requêtage d’une API, cela se fait généralement via le
**protocole HTTP**, qui est le même protocole utilisé pour charger des
pages web. Dans ce tutoriel, nous utiliserons le package `requests`, qui
simplifie le processus de requêtage et de gestion de réponses HTTP.

Les APIs que nous allons utiliser sont :

-   [Nominatim](https://nominatim.org/release-docs/latest/api/Overview/)
    : une API de géocodage proposée par **OpenStreetMap** qui nous
    permet de convertir un nom de lieu en coordonnées géographiques.
-   [Open-Meteo Weather Forecast](https://open-meteo.com/en/docs) : une
    API qui fournit des prévisions météorologiques détaillées.

## Partie 1 : récupération des coordonnées géographiques pour une localisation donnée

Avant de récupérer des données météo, nous avons besoin des coordonnées
géographiques du lieu d’intérêt. Construisons une fonction qui utilise
l’API `Nominatim` pour obtenir ces coordonnées.

In [2]:
def renvoie_lat_long(localisation):
    base_url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": localisation,
        "format": "json"
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        json_data = response.json()
        latitude = float(json_data[0]["lat"])
        longitude = float(json_data[0]["lon"])
        return (latitude, longitude)
    else:
        print(f"Erreur lors de la récupération des coordonnées de {localisation}")
        return None

Testons cette fonction sur un exemple concret :

In [3]:
LOCALISATION = "Montrouge"
lat, long = renvoie_lat_long(LOCALISATION)

print(f"Localisation : {LOCALISATION}, latitude : {lat}, longitude : {long}")

## Partie 2 : récupération des prévisions météorologiques

Maintenant que nous avons nos coordonnées, nous pouvons requêter l’API
open-meteo.com pour obtenir les données météo.

In [4]:
def fetch_meteo_data(latitude, longitude, start_date, end_date):
    url_request = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&hourly=relativehumidity_2m,windspeed_10m"
    response = requests.get(url_request)
    code = response.status_code 
    if code == 200:
        return response.json()
    else:
        print(f"Erreur lors de la récupération des données météo. Code d'erreur : {code}")
        return None

Exécutons cette fonction pour obtenir les données météo prévisionnelles
de la semaine à venir.

In [5]:
LOCALISATION = "Montrouge"
LAT, LONG = renvoie_lat_long(LOCALISATION)

data = fetch_meteo_data(LAT, LONG, START_DATE, END_DATE)

Afin de bien comprendre la structure des données que nous avons
récupérées, explorons les différents niveaux et types des clés et des
valeurs.

In [6]:
# Exploration des données
print(type(data))
print(data.keys())
print(type(data["hourly"]))
print(data["hourly"].keys())
print(type(data["hourly"]["time"]))

In [7]:
# Afficher les données
print(data['hourly']["time"][:5])
print(data['hourly']["time"][-5:])
print()
print(data['hourly']["relativehumidity_2m"][:5])
print(data['hourly']["windspeed_10m"][:5])

## Partie 3 : construction et visualisation d’un *bad hair index*

`Pandas` est une bibliothèque puissante pour la manipulation de données.
Elle facilite la gestion, la transformation et l’analyse des données.

In [8]:
import pandas as pd

In [9]:
data_dict = {
    'time': data['hourly']["time"],
    'humidity': data['hourly']["relativehumidity_2m"],
    'wind_speed': data['hourly']["windspeed_10m"]
}

df = pd.DataFrame(data_dict)
df['time'] = pd.to_datetime(df['time'])  # Convertir la colonne time en objet datetime
df.head()

Nous définissons cet indice comme le produit de l’humidité relative et
de la vitesse du vent. Il s’agit d’une mesure ludique de la probabilité
d’avoir une “mauvaise coiffure” en raison des conditions
météorologiques.

In [10]:
df['bad_hair_index'] = df['humidity'] * df['wind_speed']

df.head()

In [11]:
import seaborn as sns
import matplotlib.pyplot as plt

In [12]:
df['day'] = df['time'].dt.day
df_daily_mean = df.groupby('day').agg({'bad_hair_index': 'mean'}).reset_index()
df_daily_mean.head()

In [13]:
sns.lineplot(x='day', y='bad_hair_index', data=df_daily_mean)
plt.title("Évolution du Bad Hair Index moyen sur 7 jours")
plt.xlabel('Jour')
plt.ylabel('Bad Hair Index moyen')

In [14]:
df['hour'] = df['time'].dt.hour
df_hourly_mean = df.groupby('hour').agg({'bad_hair_index': 'mean'}).reset_index()
df_hourly_mean.head()

In [15]:
sns.lineplot(x='hour', y='bad_hair_index', data=df_hourly_mean)
plt.title("Moyenne du Bad Hair Index heure par heure sur 7 jours")
plt.xlabel('Heure')
plt.ylabel('Bad Hair Index Moyen')